<a href="https://colab.research.google.com/github/aritarodriguez/forecasting-models/blob/main/LecturaMapa_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================================
# POLÍGONO DE GUAYAQUIL - ETIQUETAR PUNTOS Y DESCARGAR RESULTADOS
# ==========================================================
!pip install geopandas shapely folium fiona fastkml -q

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from google.colab import files

# === 1️⃣ Leer los puntos del archivo Excel ===
df = pd.read_excel("TALLER FINAL.xlsx", sheet_name="Guayaquil")

# Limpiar encabezados
df.columns = df.columns.str.strip().str.upper().str.replace("\n", " ", regex=False)

# Detectar columnas con LAT / LON
col_lat = [c for c in df.columns if "LAT" in c][0]
col_lon = [c for c in df.columns if "LON" in c][0]

# Renombrar columnas
df = df.rename(columns={col_lat: "Latitud", col_lon: "Longitud"})

# Asegurar tipo numérico
df["Latitud"] = pd.to_numeric(df["Latitud"], errors="coerce")
df["Longitud"] = pd.to_numeric(df["Longitud"], errors="coerce")
df = df.dropna(subset=["Latitud", "Longitud"]).copy()

# === 2️⃣ Convertir a GeoDataFrame ===
gdf_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["Longitud"], df["Latitud"]), crs="EPSG:4326")

# === 3️⃣ Leer el polígono desde tu archivo KML ===
poly_gdf = gpd.read_file("Guayaquil.kml", driver="KML").to_crs("EPSG:4326")
poly_gdf = poly_gdf[poly_gdf.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()
poly_union = poly_gdf.unary_union

# === 4️⃣ Determinar si cada punto está dentro o fuera ===
gdf_points["Dentro_Poligono"] = gdf_points.geometry.apply(lambda p: "Sí" if poly_union.covers(p) else "No")

# === 5️⃣ Exportar resultados a Excel ===
output_excel = "puntos_con_flag.xlsx"
gdf_points.drop(columns=["geometry"]).to_excel(output_excel, index=False)

# === 6️⃣ Crear mapa Folium ===
m = folium.Map(location=[gdf_points["Latitud"].mean(), gdf_points["Longitud"].mean()],
               zoom_start=12, tiles="CartoDB positron")

# Agregar contorno del polígono
folium.GeoJson(
    poly_union,
    name="Polígono Guayaquil",
    style_function=lambda x: {"color": "#1f77b4", "weight": 2, "fillOpacity": 0.05}
).add_to(m)

# Agregar puntos (verde = dentro, rojo = fuera)
for _, row in gdf_points.iterrows():
    color = "#2ca02c" if row["Dentro_Poligono"] == "Sí" else "#d62728"
    popup_text = f"""
    <b>Sector:</b> {row.get('SECTOR','')}<br>
    <b>Local:</b> {row.get('NOMBRE DEL LOCAL','')}<br>
    <b>Lat:</b> {row['Latitud']:.5f}<br>
    <b>Lon:</b> {row['Longitud']:.5f}<br>
    <b>Dentro:</b> {row['Dentro_Poligono']}
    """
    folium.CircleMarker(
        location=[row["Latitud"], row["Longitud"]],
        radius=4,
        color=color,
        fill=True,
        fill_opacity=0.8,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

# Guardar mapa
output_map = "mapa_validacion.html"
m.save(output_map)

# === 7️⃣ Descargar ambos archivos ===
files.download(output_excel)
files.download(output_map)

print("✅ Proceso completado. Archivos descargados.")
m


/tmp/ipython-input-1393582203.py:36: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  poly_union = poly_gdf.unary_union


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Proceso completado. Archivos descargados.


In [ ]:
# ==========================================================
# CLUSTERING GEOGRÁFICO CON K-MEANS (Geocercas 50-150 clientes)
# ==========================================================
!pip install pandas scikit-learn folium openpyxl -q

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
import math

# === 1️⃣ Leer el archivo con puntos geográficos ===
df = pd.read_excel("puntos_con_flag.xlsx")

# Asegurar columnas de coordenadas
df.columns = df.columns.str.strip().str.upper()
col_lat = [c for c in df.columns if "LAT" in c][0]
col_lon = [c for c in df.columns if "LON" in c][0]
col_id  = [c for c in df.columns if "N" in c][0]  # Columna ID del cliente (N°)

df = df.rename(columns={col_lat: "Latitud", col_lon: "Longitud", col_id: "ID"})
df = df.dropna(subset=["Latitud", "Longitud"]).copy()

# === 2️⃣ Determinar número óptimo de clústeres (entre 50-150 clientes por geocerca) ===
num_clientes = len(df)
clientes_por_cluster = 100  # promedio deseado
k = max(1, round(num_clientes / clientes_por_cluster))

print(f"Total clientes: {num_clientes}")
print(f"Número de clústeres estimado: {k}")

# === 3️⃣ Ejecutar K-Means ===
X = df[["Latitud", "Longitud"]].values
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
df["Cluster"] = kmeans.fit_predict(X)

# === 4️⃣ Calcular centroides ===
centroids = pd.DataFrame(kmeans.cluster_centers_, columns=["Latitud", "Longitud"])
centroids["Cluster"] = range(k)

# === 5️⃣ Crear mapa Folium ===
map_center = [df["Latitud"].mean(), df["Longitud"].mean()]
m = folium.Map(location=map_center, zoom_start=11, tiles="CartoDB positron")

# Colores distintos por cluster
import random
random.seed(42)
colors = [f"#{random.randint(0,0xFFFFFF):06x}" for _ in range(k)]

for i, row in df.iterrows():
    c = colors[row["Cluster"] % len(colors)]
    popup_text = f"""
    <b>ID:</b> {row['ID']}<br>
    <b>Cluster:</b> {row['Cluster']}<br>
    <b>Latitud:</b> {row['Latitud']:.5f}<br>
    <b>Longitud:</b> {row['Longitud']:.5f}
    """
    folium.CircleMarker(
        location=[row["Latitud"], row["Longitud"]],
        radius=3,
        color=c,
        fill=True,
        fill_opacity=0.7,
        popup=popup_text
    ).add_to(m)

# Agregar los centroides
for _, row in centroids.iterrows():
    folium.Marker(
        location=[row["Latitud"], row["Longitud"]],
        icon=folium.Icon(color="red", icon="flag", prefix="fa"),
        popup=f"Centro Cluster {row['Cluster']}"
    ).add_to(m)

# Guardar mapa
m.save("mapa_clusters.html")

# === 6️⃣ Guardar resultados ===
output_excel = "clientes_clusterizados.xlsx"
df.to_excel(output_excel, index=False)

print("✅ Clustering completado.")
print(f"Archivo generado: {output_excel}")
print("Mapa interactivo: mapa_clusters.html")

# Mostrar mapa en Colab
m


Total clientes: 725
Número de clústeres estimado: 7
✅ Clustering completado.
Archivo generado: clientes_clusterizados.xlsx
Mapa interactivo: mapa_clusters.html
